Most of the code below is adapted from code published in Swanson et al. 2024.
https://github.com/swansonk14/SyntheMol/blob/main/docs/antibiotics.md

In [ ]:
# Packages installed in new Python 3.11 venv
!pip install synthemol
!pip install scikit-learn==1.6.0 #Remove error related to deprecated variable
!pip install seaborn
!pip install matplotlib

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Import docking scores
df_docking = pd.read_csv("docking_results.csv")
df_docking

smiles  TotalEnergy
0                  Br.COC=1C=CC(CCN=C2NCCCCN2)=C(Cl)C1OC    -105.1413
1               BrC1=CC=C(O1)C2=NOC(=N2)C3=CC=NN3C4CCCC4    -111.8644
2                    BrC1=CSC(=C1)C2=NOC(CSC3=NN=CN3)=N2    -113.7583
3      BrC=1C=CC(=CC1)C2(CNC(C3=NC=CN3)C=4C=CC=NC4)CC...    -115.4544
4            BrC=1C=CC(=CC1)C2=NNC=C2CN3CCN(CC3)C4CCOCC4    -111.6539
...                                                  ...          ...
49704  [O-][N+](=O)C=1C=CC=2N=C(CN3CCOC4(C3)CCCCCC4)N...    -104.1580
49705  [O-][N+](=O)C=1C=CC=C(C1)C(=O)N2CCN(CC2)C3C4=C...    -111.0011
49706  [O-][N+](=O)C=1C=CC=C(C1)C2=NC(=CN2)C(=O)NC3=N...    -128.3897
49707  [O-][N+](=O)C=1C=CC=C2C1CCN2C(=O)C=3C=CC(=O)N(...    -116.5066
49708  [O-][N+](=O)C=1C=CC=CC1C(=O)NC2=C3CCCC3=NN2C=4...    -116.1878

[49709 rows x 2 columns]

In [6]:
# Remove entries with docking TotalEnergy > 0 as outliers
df_docking_filtered = df_docking[df_docking['TotalEnergy'] <= 0]
df_docking_filtered.to_csv("docking_filtered.csv", index=False)

In [10]:
# Save docking hits as top 500 minimum energy
df_docking_hits = df_docking_filtered.nsmallest(500, 'TotalEnergy')
df_docking_hits.to_csv("docking_hits.csv", index=False)

In [4]:
# Establish separate datasets for training/tuning and final model testing
df_training = df_docking_filtered.sample(frac = 0.9)
df_testing = df_docking_filtered.drop(df_training.index)

df_training.to_csv("training_data.csv", index=False)
df_testing.to_csv("testing_data.csv", index=False)

In [ ]:
# 5-fold cross validation of the random forest architecture - run in external command line
python scripts/train.py \
    --data_path training_data.csv \
    --save_dir models/random_forest \
    --model_type random_forest \
    --dataset_type regression \
    --fingerprint_type rdkit \
    --property_column TotalEnergy \
    --num_models 5 \
    --epochs 30 \
    --num_workers 8

In [ ]:
# 5-fold cross validation of the chemprop-rdkit architecture - run in external command line
python scripts/train.py \
    --data_path training_data.csv \
    --save_dir models/chemprop_rdkit_20epoch \
    --model_type chemprop \
    --dataset_type regression \
    --fingerprint_type rdkit \
    --property_column TotalEnergy \
    --num_models 5 \
    --epochs 20 \
    --num_workers 8

Best epochs (minimum validation loss) were 16, 19, 19, 15, 19 for model_0 through model_5, respectively. The MAE of chemprop-rdkit is ~0.5 better than random forest, so this architecture was chosen. 5-Fold cross validation was repeated using 30 epochs.

In [16]:
# Import 5-fold validation results
df_chemprop_20epoch = pd.read_csv('models/chemprop_rdkit_20epoch/scores.csv')
df_chemprop_20epoch['model'] = 'chemprop_rdkit'
df_random_forest = pd.read_csv('models/random_forest/scores.csv')
df_random_forest['model'] = 'random_forest'
df_5fold_validation = pd.concat([df_chemprop_20epoch, df_random_forest], ignore_index=True)

In [17]:
mae_means = df_5fold_validation.groupby('model')['mae'].mean()
r2_means = df_5fold_validation.groupby('model')['r2'].mean()
print(mae_means)
print(r2_means)

model
chemprop_rdkit    6.797798
random_forest     7.312264
Name: mae, dtype: float64
model
chemprop_rdkit    0.455398
random_forest     0.368844
Name: r2, dtype: float64


In [95]:
# Plot 5-fold validation results
fig, ax = plt.subplots(1, 2, figsize=(8,4))
sns.stripplot(data = df_5fold_validation, x = 'model', y = 'mae', hue = 'model', ax = ax[0], legend = False)
ax[0].annotate(6.798, ('chemprop_rdkit', 9), ha="center")
ax[0].annotate(7.312, ('random_forest', 9), ha="center")
ax[0].set_title('Mean Absolute Error')
ax[0].set_ylim(0,10)
ax[0].set(ylabel='', xlabel='')
sns.stripplot(data = df_5fold_validation, x = 'model', y = 'r2', hue = 'model', ax = ax[1])
ax[1].annotate(0.455, ('chemprop_rdkit', 0.9), ha="center")
ax[1].annotate(0.369, ('random_forest', 0.9), ha="center")
ax[1].set_title('$R^2$')
ax[1].set_ylim(0,1)
ax[1].set(ylabel='', xlabel='')
plt.savefig('5-fold_validation', bbox_inches='tight')
plt.show()

In [ ]:
# 5-fold cross validation of the chemprop-rdkit architecture - run in external command line
python scripts/train.py \
    --data_path training_data.csv \
    --save_dir models/chemprop_rdkit_30epoch \
    --model_type chemprop \
    --dataset_type regression \
    --fingerprint_type rdkit \
    --property_column TotalEnergy \
    --num_models 5 \
    --epochs 30 \
    --num_workers 8

A memory error occurred after 2/5 models were trained. The best epochs were 26 and 28 for model_0 and model_1, respectively. The MAE did not decrease relative to 20 epochs, so the full 5-fold validation was not repeated. 30 epochs was chosen for training the final model as performance appears to plateau.

In [ ]:
# Run modified script to train model on all training data and test on holdout test data - run in external command line
python scripts/train_final.py \
    --data_path training_data.csv \
    --save_dir models/chemprop_rdkit_final \
    --model_type chemprop \
    --dataset_type regression \
    --fingerprint_type rdkit \
    --property_column TotalEnergy \
    --num_models 1 \
    --epochs 30 \
    --num_workers 8

The model was trained successfully with MAE = 6.702 and R^2 = 0.466, an improvement over the 5-fold validation runs.

In [ ]:
# Run pre-predictions on building blocks - run in external command line
python scripts/predict.py \
    --data_path building_blocks.csv \
    --save_path building_blocks.csv \
    --model_path models/chemprop_rdkit_final \
    --model_type chemprop \
    --fingerprint_type rdkit \
    --smiles_column smiles

In [ ]:
# Run molecule generation for 20000 iterations with up to 2 reactions per molecule - run in external command line
synthemol \
    --model_path models/chemprop_rdkit_final \
    --model_type chemprop \
    --building_blocks_path building_blocks.csv \
    --building_blocks_score_column pred_TotalEnergy \
    --building_blocks_id_column Reagent_ID \
    --reaction_to_building_blocks_path reaction_to_building_blocks_filtered.pkl \
    --save_dir generation \
    --fingerprint_type rdkit \
    --max_reactions 2 \
    --n_rollout 20000 \
    --optimization minimize

55593 molecules were generated in 6 hours.

In [ ]:
# Calculate Tversky similarity between the generated molecules and the top 500 docking hits - run in external command line
chemfunc nearest_neighbor \
    --data_path generation/molecules.csv \
    --reference_data_path docking_hits.csv \
    --reference_name docking \
    --metric Tversky

In [ ]:
# Filter by Tversky similarity <=0.5 to avoid repetitive molecules - run in external command line
chemfunc filter_molecules \
    --data_path generation/molecules.csv \
    --save_path generation/molecules_sim_below_0.5.csv \
    --filter_column docking_tversky_nearest_neighbor_similarity \
    --max_value 0.5

In [ ]:
# Filter by top 20% predicted docking hits (lowest TotalEnergy) - run in external command line
chemfunc filter_molecules \
    --data_path generation/molecules_sim_below_0.5.csv \
    --save_path generation/molecules_sim_below_0.5_top_20.csv \
    --filter_column score \
    --bottom_proportion 0.2

In [ ]:
# Use K-means clustering on Morgan fingerprints to define 50 clusters - run in external command line
chemfunc cluster_molecules \
    --data_path generation/molecules_sim_below_0.5_top_20.csv \
    --num_clusters 50

In [19]:
# Filter for the top predicted hit of each cluster
df_cluster_mol = pd.read_csv('generation/molecules_sim_below_0.5_top_20.csv')
df_best_mol = df_cluster_mol.loc[df_cluster_mol.groupby('cluster_label')['score'].apply(lambda x: x.idxmin())]
df_best_mol.to_csv("generation/molecules_sim_below_0.5_top_20_selected.csv", index=False)